# Develop an NN Solution 🧠

Apply your PyTorch skills to the problem of optimized battery charging; training a simple NN to predict the `duration` for how long a user will plug-in and charge their device in future, based on past data. 

At this point, it is expected that you have two featurized, battery charging datasets: training data and test data. In this notebook, you should load in that data and use it to train and evaluate a simple neural network. 

This neural network does not need to be your best solution, just a proof-of-concept. 

> To know whether or not you are on the right track, aim for a test RMSE of around 3hrs. 

This RMSE (the square root of a squared value) roughly means that the average error in predicting the duration of a plug-in charge event is around 3hrs. The ideal would be 0 average error, but the reality is that being within a few hours of a correct prediction can still be very helpful in deciding when to pause and resume charging, such that battery lifespan can be increased without a sacrifice to the user experience! 

### Your tasks
To create an NN-based solution to OBC, complete the following tasks:

**Part 1: Loading Custom Data**

1. Load in your train/test battery datasets
2. Create DataLoaders for those datasets

**Part 2: Training and Evaluating a Simple NN**

3. Define and train a simple NN
4. Evaluate your NN on some test data, recording the resultant RMSE

You may be wondering: *How do I submit this in parts?* 

> You will be expected to submit this notebook **twice** for grading; once, when you've completed part one, and once when you've completed the entire baseline solution (part two). 

**Hint**: It may be helpful to reference the codebase for Household Power Prediction, which you saw a while ago—this codebase contains several helper functions for training and testing a model; including converting a typical MSE function into an RMSE value. 

In [61]:
# Import Nessecary Libraries
import torch
import torch.nn as nn
import torch.optim as optim
import sklearn
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import StandardScaler
from torch.nn import MSELoss
from torch.utils.data import Dataset, DataLoader, random_split
import pandas as pd

In [62]:
class OBC_Dataset(Dataset):
    def __init__(self, filename):
        # Read-in DataFrame
        df = pd.read_csv(filename)

        # Separate features and target (target = last column)
        input_features = df.iloc[:, :-1].values
        target = df.iloc[:, -1:].values
        
        # Convert features and target into tensors
        self.x = torch.tensor(input_features, dtype=torch.float32)
        self.y = torch.tensor(target, dtype=torch.float32)

    def __len__(self):
        # Necessary __len__ method
        return len(self.y)

    def __getitem__(self, index):
        # Necessary fetch item method
        return self.x[index], self.y[index]

    def split_data(self, n_test):
        # Split into desired train-test split
        test_size = round(n_test * len(self.x))
        train_size = len(self.x) - test_size
        return random_split(self, [train_size, test_size])

In [63]:
# Get DataFrame
df = OBC_Dataset("data/scaled_filtered_dataset.csv")

In [64]:
# Split Into Train-Test Split
train, test = df.split_data(n_test = 0.2)

In [65]:
# Examine Train Length to Ensure Validity
len(train)

1022650

In [66]:
# Examine Test Length to Ensure Validity
len(test)

255662

In [67]:
# Examine Input Features & Target of Sample
index = 2
features, target = train[index]
print("Features at position: ", index, ":", features)
print("\n\nTarget at position: ", index, ":", target)

Features at position:  2 : tensor([  1.0000,   2.0000,  20.0000,   1.8500,   0.0000,  10.0000, 199.2039,
          6.0000])


Target at position:  2 : tensor([0.2200])


In [68]:
# Add to Data Loaders
train_loader = DataLoader(train, batch_size = 64, shuffle = True)
test_loader = DataLoader(test, batch_size = 64)
# ------- END OF ASSIGNMENT ONE ------- #

# Develop A Neural Network Solution: Part Two 🛠️

In [69]:
# Define Model Structure
class Construct_Model(nn.Sequential):
    def __init__(self, input_dim, hidden_dimension_1, hidden_dimension_2, output_dim):
        super(Construct_Model, self).__init__(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )

# Define Dimensions
input_dim = 8
hidden_1 = 256
hidden_2 = 128
output_dim = 1

# Device setup for compatibility with GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define Loss Function, Construct Model, Epochs, & Define Optimizer
criterion = nn.MSELoss()
model = Construct_Model(input_dim, hidden_1, hidden_2, output_dim).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.00001)
num_epochs = 10

# Train The Model
model.train()
for epoch in range(1, num_epochs + 1):
    train_loss = 0.0
    for data, target in train_loader:
        # Move data and target to the same device as model
        data, target = data.to(device), target.to(device)
        
        # Forward pass
        optimizer.zero_grad()
        output = model(data)
        
        # Compute loss
        loss = criterion(output, target)
        loss.backward()
        
        # Update weights
        optimizer.step()
        
        # Accumulate training loss
        train_loss += loss.item()
    
    # Print epoch training loss
    print(f"Epoch: {epoch}, Loss: {train_loss / len(train_loader)}")

Epoch: 1, Loss: 95400.25622937533
Epoch: 2, Loss: 1549.241419843305
Epoch: 3, Loss: 990.7338023949178
Epoch: 4, Loss: 661.54464214656
Epoch: 5, Loss: 514.5609981297339
Epoch: 6, Loss: 412.4138351629852
Epoch: 7, Loss: 342.6434326548913
Epoch: 8, Loss: 259.8928284798867
Epoch: 9, Loss: 251.4414346831818
Epoch: 10, Loss: 202.8594396838882


In [71]:
model.eval()
def evaluate_model(model, test_loader, criterion):
    test_loss = 0.0
    num_batches = 0
    
    with torch.no_grad():  # Disable gradient computation
        for data, target in test_loader:
            # Move data to the same device as model
            data, target = data.to(device), target.to(device)
            
            # Get model predictions
            output = model(data)
            
            # Calculate batch loss
            loss = torch.sqrt(criterion(output, target))
            test_loss += loss.item()
            num_batches += 1
    
    # Calculate average loss
    avg_loss = test_loss / num_batches
    
    return avg_loss

In [81]:
# Fetch RSME
test_rmse = evaluate_model(model, test_loader, criterion)

# Multiply by 100 to re-scale data
print("Our RMSE is:", test_rmse, "; indicating that on average, our prediction is off by", round((test_rmse * 100) / 60), "hours. 🥳")

Our RMSE is: 1.1711138074478609 ; indicating that on average, our prediction is off by 2 hours. 🥳
